# Programowanie sieciowe
## Serwer TCP
Aby stworzyć prosty serwer echo TCP, możemy posłużyć się modułem biblioteki standardowej `socketserver`:

In [ ]:
from socketserver import BaseRequestHandler, TCPServer

LISTEN_PORT = 8888

class Echo(BaseRequestHandler):
    def handle(self):
        print('Connected from ', self.client_address)
        while True:
            msg = self.request.recv(8192)
            if not msg:
                break
            self.request.send(msg)

if __name__ == '__main__':
    serv = TCPServer(('', LISTEN_PORT), Echo)
    serv.serve_forever()

Możemy sprawdzić interakcję z nim przy użyciu następującego kodu, działającego jako osobny interpreter:

In [ ]:
from socket import socket, AF_INET, SOCK_STREAM

LISTEN_PORT = 8888

s = socket(AF_INET, SOCK_STREAM)
s.connect(('localhost', LISTEN_PORT))
s.send(b'Hello')
s.recv(8192)

Taki serwer jest oczywiście jednowątkowy i bardzo prymitywny - zazwyczaj interesuje nas wydajniejsza implementacja umożliwiająca zrównoleglenie obsługi zapytań, kontrolę nad tą równoległością, obsługę protokołu HTTP etc. Aby to osiągnąć najlepiej jest użyć gotowego serwera WSGI - np.: Gunicorn albo Uvicorn oraz sensownego frameworka HTTP, których w Pythonie nie brakuje. Większość z nich pod spodem korzysta np.: z poznanej tu biblioteki `socketserver`, jednak wiele problemów jest już rozwiązanych przez specjalistów w tej dziedzinie, którzy kontrybuują do projektów Open Source w ekosystemie Pythona.

## Eksponowanie API HTTP

### WSGI - [PEP-3333](https://www.python.org/dev/peps/pep-3333/)
Interfejs dla webserwera
```
def application(environ, start_response):
    body = b'Hello world!\n'
    status = '200 OK'
    headers = [('Content-type', 'text/plain')]
    start_response(status, headers)
    return [body]
```


### Flask
Do stworzenia API HTTP posłużymy się frameworkiem `Flask`. W ekosystemie Pythona jest wiele różnych bibliotek i frameworków, z których można skorzystać - najpopularniejszym jest `Django`, a w szczególności jego rozszerzenie `Django Rest Framework`. `Django` jest również najbardziej rozbudowane, posiada gotowe rozwiązania dla wielu standardowych problemów związanych z budowaniem aplikacji webowych - template'y, ORM, migracje etc. Filozofią `Django` jest wymuszać określoną strukturę projektu i określone, uważane przez jego twórców za prawidłowe wzorce budowania aplikacji.
`Flask` dla odmiany stara się być możliwie modularny, niewiele narzucając i będąc przez to dużo łatwiejszym do nauki. Jego popularność również jest duża, lata rozwoju skutkują dobrą stabilnością, jednak w niektórych aspektach wymaga on bardziej bezpośredniego podejścia do rozwiązywanych problemów. Do naszych zastosowań będzie jednak lepszy, ponieważ jego elastyczność umożliwi nam łatwe rozwinięcie `dirwatchera` o API HTTP, bez konieczności uczenia się wszystkich konceptów wymaganych przez `Django`. 

Najprostsza aplikacja we `Flask`u wygląda następująco:
```
from flask import Flask
app = Flask(__name__)

@app.route('/', methods=['GET'])
def hello_world():
    return 'Hello, World!'
```
Po zaimportowaniu wystaczy utworzyć obiekt reprezentujący naszą aplikację, a następnie użyć dekoratora `@app.route` by zamienić funkcję zwracającą jakiś wynik na endpoint HTTP dostępny pod ścieżką podaną w argumencie dekoratora.
Aby uruchomić wbudowany we Flaska serwer HTTP (wyłącznie do celów developerskich!), należy ustawić zmienną środowiskową `FLASK_APP` na ścieżkę do pliku z naszym kodem i wywołać `flask run`:
```
>>> export FLASK_APP=hello.py # na uniksach
>>> set FLASK_APP=hello.py # w windowsowym cmd
>>> $env:FLASK_APP = "hello.py" # w powershell
>>> flask run
 * Running on http://127.0.0.1:5000/
```
Domyślnie `Flask` wystawi nasz endpoint pod adresem `http://127.0.0.1:5000/`.

Przydatne w przypadku kłopotów jest dodatkowe ustawienie lokalnie zmiennej `FLASK_ENV=development`, co pozwoli nam dostać lepsze informacje o błędach, dostępne również przez przeglądarkę. 
[Dokumentacja Flaska](https://flask.palletsprojects.com/en/2.0.x/quickstart/) zawiera więcej informacji n.t. tego jak szybko zacząć korzystać z tego narzędzia, dlatego nie będzie tu przepisywana.


### *Zadanie 1*
```
git checkout task-8
git checkout -b my-solution-8
```
* [ ] Utwórz handler `has_anything changed`, monitorujący katalog podany w url:
    - utwórz nową instancję usługi `WatcherService`
    - wywołaj metodę `has_anything_changed` z usługi `WatcherService`
    - obsłuż możliwe wyjątki i zwróć `400` w przypadku wystąpienia wyjątku `NoPriorCheckpointSavedError`
    - zwróć wartość typu `bool`
    - endpoint powinien być dostępny pod adresem `/<nazwa_podfolderu>/ischanged`
    - użyj metody HTTP `GET`
* [ ] Napisz test integracyjny testujący napisany endpoint

### *Zadanie 2*
- [ ] Utwórz endpoint `list_deleted`, zwracający listę plików skasowanych od ostatniego sprawdzenia
    - użyj metody `get_changes_since_last_checkpoint` z `WatcherService` by dostać listę skasowanych plików
    - obsłuż możliwe wyjątki i zwróć `400` w przypadku wystąpienia wyjątku `NoPriorCheckpointSavedError`
    - endpoint powinien być dostępny pod adresem `/<nazwa_podfolderu>/deleted`
    - użyj metody HTTP `GET`
    - wynik powinien być zserializowany jako lista stringów w formacie JSON
    - zaproponuj analogiczne endpointy dla nowych i zmienionych plików

